## CSV dosyasındaki verinin yüklenmesini gerçekleştirme

In [27]:

from langchain_community.document_loaders.csv_loader import CSVLoader

filepath = "IMDb_Top_1000_Movies_Dataset.csv"
loader = CSVLoader(filepath, encoding="utf-8", source_column="Movie_Name")

data = loader.load()

print(f"Toplam {len(data)} film yüklendi.")

Toplam 1000 film yüklendi.


## Veriyi parçalarına ayırma(Chunking işlemi)


In [29]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=0)
docs = text_splitter.split_documents(data)

print(f"Toplam {len(docs)} film parçası oluşturuldu.")

Toplam 1000 film parçası oluşturuldu.


## Google Generative AI Embeddings'i kullanarak Embedding Oluşturma İşlemi


In [30]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv


In [31]:
load_dotenv()  # Load environment variables from .env file
import os
api_key = os.getenv("GOOGLE_API_KEY")
if api_key is None:
    print("HATA: API anahtarı .env dosyasından okunamadı!")
else:
    print(f"Anahtar başarıyla yüklendi. İlk 4 karakter: {api_key[:4]}")

Anahtar başarıyla yüklendi. İlk 4 karakter: AIza


In [32]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
vector = embeddings.embed_query("hello, world!")
vector[:5]

[0.014134909, -0.022324152, -0.05460342, -0.0062845494, -0.033924025]

## ChromaDB üzerine kayıt işlemi

In [46]:
from langchain_chroma import Chroma

In [47]:
vector_store = Chroma.from_documents(documents=docs, embedding=embeddings)

In [48]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 10 })

In [49]:
retrieved_docs = retriever.invoke("Movies starring Meryl Streep")     

len(retrieved_docs)

print(retrieved_docs[1].page_content)


: 973
Movie_Name: The Bridges of Madison County
Movie_Rating: 7.6
Movie_Certificate: A
Movie_Year: 1995
Movie_Runtime: 135 min
Movie_Cast: Clint Eastwood, Meryl Streep, Annie Corley, Victor Slezak
Movie_Genre: Drama, Romance
Movie_Poster: https://m.media-amazon.com/images/M/MV5BNmRiZDZkN2EtNWI5ZS00ZDg3LTgyNDItMWI5NjVlNmE5ODJiXkEyXkFqcGdeQXVyMjQwMjk0NjI@._V1_UX67_CR0,0,67,98_AL_.jpg
Movie_Description: Photographer Robert Kincaid wanders into the life of housewife Francesca Johnson for four days in the 1960s.
Movie_Poster_HD: https://m.media-amazon.com/images/M/MV5BNmRiZDZkN2EtNWI5ZS00ZDg3LTgyNDItMWI5NjVlNmE5ODJiXkEyXkFqcGdeQXVyMjQwMjk0NjI@._V1_.jpg
All_Movie_Info: the bridges of madison county clint eastwood meryl streep annie corley victor slezak drama romance photographer robert kincaid wanders into the life of housewife francesca johnson for four days in the 1960s


## Google Gemini API Yapısını Kullanarak LLM Tetikleme İşlemleri

- Düşük Değerler (0.1-0.4): Daha kesin ve daha tutarlı cevaplar verilir. Model daha tahmin edilebilir hale gelir.
- Orta Değerler (0.5-0.7): Hem mantıklı hem de yaratıcı cevaplar verilir.
- Yüksek Değerler (0.7-1): Daha rastgele ve yaratıcı, ancak bazen tutarsız yanıtlar verebilir.

In [62]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",  # Gemini 2.5 model
    temperature=0.3,  # Gemini 3.0+ defaults to 1.0
    max_tokens=500,
)

In [63]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain


In [64]:
system_prompt = (
    "You are a helpful movie recommendation assistant. Use the following movie database context to answer the user's question."
    "Only recommend movies from the provided context"
    "\n"
    "\n"
    "If you don't find suitable movies in the context, say so honestly."
    "\n\n"
    "{context}"
)



In [65]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}"),
    ]
)


## Soru-Cevap Zinciri Oluşturma (LLM + PROMPT)


In [66]:
question_answering_chain = create_stuff_documents_chain(llm, prompt)

## RAG Zinciri Oluşturma (RAG + LLM)


In [67]:
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

## Kullanıcı sorgusunu çalıştırma

In [68]:
response = rag_chain.invoke({"input": "Suggest comedy movies under 90 minutes"})

print(response["answer"])

KeyboardInterrupt: 